In [1]:
! pip install pandas
! pip install seaborn


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Célula 1 — Imports e paths
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

root = Path('notebook_Parte2').resolve().parents[2]  # ajuste se seu DataLake estiver em outro local
raw_a2 = root / 'raw' / 'atividade2'
bronze_a2 = root / 'bronze' / 'atividade2'
gold_a2 = root / 'gold' / 'atividade2'
code_a2 = root / 'code' / 'atividade2'
meta_a2 = root / 'metadata' / 'atividade2'

for p in [raw_a2, bronze_a2, gold_a2, code_a2, meta_a2]:
    p.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({'figure.max_open_warning': 0})

In [3]:
# Célula 2 — Funções utilitárias
def save_parquet(df, path: Path, metadata: dict = None):
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(path, index=False)
    if metadata is not None:
        meta_path = path.with_suffix('.json')
        meta_path.write_text(pd.Series(metadata).to_json())  # simple json

def read_csv_flexible(path: Path):
    return pd.read_csv(path, low_memory=False)

In [4]:
# Caminhos
historico_path = raw_a2/ "olympics_historico.csv"
output_path = raw_a2/ "olympics_historico_ajustado.csv"

# Leitura
df = pd.read_csv(historico_path, low_memory=False)

# --- Verificações ---
if not {'is_team_sport', 'year', 'event', 'country', 'medal'}.issubset(df.columns):
    raise ValueError("O dataset precisa conter as colunas: is_team_sport, year, event, country, medal")

# Normalizar campos
df['is_team_sport'] = df['is_team_sport'].astype(str).str.lower().isin(['true', '1'])
df['year'] = df['year'].astype(str).str.strip()
df['event'] = df['event'].astype(str).str.strip()
df['country'] = df['country'].astype(str).str.strip()
df['medal'] = df['medal'].fillna("").astype(str).str.strip()

# --- Separar esportes de time e individuais ---
df_team = df[df['is_team_sport'] == True].copy()
df_indiv = df[df['is_team_sport'] == False].copy()

# 🔹 Para esportes de time:
# Manter apenas uma medalha por país + edição + evento + medalha
df_team_unique = df_team.drop_duplicates(subset=['year', 'event', 'country', 'medal'])

# 🔹 Reunir novamente os dois conjuntos
df_adjusted = pd.concat([df_indiv, df_team_unique], ignore_index=True)

# Resetar índices e limpar
df_adjusted = df_adjusted.reset_index(drop=True)

# Salvar resultado
df_adjusted.to_csv(output_path, index=False)

print(f"Arquivo ajustado salvo em: {output_path}")
print(f"Linhas originais: {len(df)} → Linhas após ajuste: {len(df_adjusted)}")
print(f"Medalhas de time agora contabilizadas apenas uma vez por evento e país.")

Arquivo ajustado salvo em: D:\olympics-datalake\raw\atividade2\olympics_historico_ajustado.csv
Linhas originais: 316834 → Linhas após ajuste: 219158
Medalhas de time agora contabilizadas apenas uma vez por evento e país.


In [5]:
# Caminho do arquivo de entrada e saída
paris_path = raw_a2 / "olympics_paris2024.csv"
output_path = raw_a2/ "olympics_paris2024_ajustado.csv"

# Ler CSV
df = pd.read_csv(paris_path, low_memory=False)

# --- Verificações básicas ---
required_cols = {'event', 'country', 'team', 'medal_type'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Colunas ausentes no dataset: {missing}")

# --- Normalizações ---
df['team'] = df['team'].fillna("").astype(str).str.strip()
df['event'] = df['event'].fillna("").astype(str).str.strip()
df['country'] = df['country'].fillna("").astype(str).str.strip()
df['medal_type'] = df['medal_type'].fillna("").astype(str).str.strip()

# --- Separar eventos individuais e coletivos ---
df_team = df[df['team'] != ""].copy()       # há time (esporte coletivo)
df_indiv = df[df['team'] == ""].copy()      # sem time (esporte individual)

# 🔹 Para eventos coletivos:
# Mantém apenas uma linha por país + evento + tipo de medalha
df_team_unique = df_team.drop_duplicates(subset=['team', 'event', 'country', 'medal_type'])

# 🔹 Reunir novamente
df_adjusted = pd.concat([df_indiv, df_team_unique], ignore_index=True)

# Resetar índices
df_adjusted = df_adjusted.reset_index(drop=True)

# Salvar CSV final ajustado
df_adjusted.to_csv(output_path, index=False)

print(f"Arquivo ajustado salvo em: {output_path}")
print(f"Linhas originais: {len(df)} → Linhas após ajuste: {len(df_adjusted)}")
print("Medalhas de time agora contabilizadas apenas uma vez por evento e país.")


Arquivo ajustado salvo em: D:\olympics-datalake\raw\atividade2\olympics_paris2024_ajustado.csv
Linhas originais: 11374 → Linhas após ajuste: 10094
Medalhas de time agora contabilizadas apenas uma vez por evento e país.


In [6]:
# Célula 3 — Carregar dados raw (usuário coloca os CSVs em raw/atividade2/)
# Ajuste os nomes dos arquivos conforme os seus arquivos locais.
hist_path = raw_a2 / 'olympics_historico_ajustado.csv'          # historical (Base dos Dados)
paris_path = raw_a2 / 'olympics_paris2024_ajustado.csv'         # Paris 2024 merged (ou use athletes+medallists)
# Caso você já possua bronze/parquet, também pode apontar para eles
if hist_path.exists():
    df_hist = read_csv_flexible(hist_path)
else:
    raise FileNotFoundError(f"{hist_path} não encontrado — coloque o CSV em raw/atividade2/")

if paris_path.exists():
    df_paris = read_csv_flexible(paris_path)
else:
    # permite que usuário tenha athletes.csv + medallists.csv — você pode adaptar conforme seu caso
    p1 = raw_a2 / 'athletes.csv'
    p2 = raw_a2 / 'medallists.csv'
    if p1.exists() and p2.exists():
        athletes = read_csv_flexible(p1)
        med = read_csv_flexible(p2)
        # replicar a lógica de expansão de medalhas (uma linha por medalha)
        athletes['code'] = athletes['code'].astype(str).str.strip()
        med['code_athlete'] = med['code_athlete'].astype(str).str.strip()
        med = med[['code_athlete', 'medal_type', 'sport', 'event', 'medal_date']].copy()
        med['medal_type'] = med['medal_type'].astype(str).str.replace(' Medal','', regex=False).str.strip()
        df_paris = athletes.merge(med, left_on='code', right_on='code_athlete', how='left')
        df_paris['year'] = 2024
    else:
        raise FileNotFoundError("Nenhum arquivo Paris encontrado (paris csv ou athletes+medallists).")


In [7]:

# 2) Limpeza básica e harmonização de colunas
def harmonize(df):
    df = df.copy()
    # padronizar nomes de colunas comuns
    colmap = {}
    for c in df.columns:
        lc = c.strip().lower()
        if lc in ['year','edition']:
            colmap[c] = 'year'
        if lc in ['city','host_city']:
            colmap[c] = 'city'
        if lc in ['sport','sports']:
            colmap[c] = 'sport'
        if lc in ['event','event_name', 'medal_events', 'events']:
            colmap[c] = 'event'
        if lc in ['athlete','name']:
            colmap[c] = 'athlete'
        if lc in ['country']:
            colmap[c] = 'country_long'
        if lc in ['country_code','country_noc']:
            colmap[c] = 'country'
        if lc in ['medal','medals', 'medal_type']:
            colmap[c] = 'medal'
        if lc in ['sex','gender']:
            colmap[c] = 'sex'
    df = df.rename(columns=colmap)
    # keep essential columns if exist
    expected = ['year','city','sport','event','athlete','country', 'country_long', 'medal','sex', 'season']
    for e in expected:
        if e not in df.columns:
            df[e] = pd.NA
    # normalize year to int where possible
    df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
    # standardize sex values
    df['sex'] = df['sex'].astype('string').str.upper().replace({'M':'M','F':'F','MALE':'M','FEMALE':'F'})

    # trim strings
    strcols = ['city','sport','event','athlete','country', 'country_long', 'medal','sex', 'season']
    for c in strcols:
        if c in df.columns:
            df[c] = df[c].astype('string').str.strip()
    return df

# Apply harmonization if files exist
if 'df_hist' in globals():
    df_hist = harmonize(df_hist)
    df_paris = harmonize(df_paris)
    print('Harmonização concluída. Exemplo colunas:', df_hist.columns.tolist())


Harmonização concluída. Exemplo colunas: ['edition_id', 'country', 'sport', 'event', 'result_id', 'athlete', 'athlete_id', 'position', 'medal', 'is_team_sport', 'sex', 'country_long', 'year', 'season', 'medal_count', 'city']


In [8]:
# Célula 5 — Filtrar período 1986–2024 e concat
df_all = pd.concat([df_hist, df_paris], ignore_index=True, sort=False)
df_all = df_all[df_all['year'].notna()]
df_all = df_all[(df_all['year'] >= 1896) & (df_all['year'] <= 2024)]  # manter todo histórico, ajustar se quiser 1986+
df_all.reset_index(drop=True, inplace=True)

# Salvar bronze parquet integrado
save_parquet(df_all, bronze_a2 / 'olympics_merged_1896_2024.parquet',
            metadata={'dataset':'olympics_merged','rows':len(df_all),'columns':list(df_all.columns)})
print("Linhas df_all:", len(df_all))


Linhas df_all: 229252


# Parte 1: Consolidação de Medalhas por País

In [9]:
# Célula 6 — Preparar dataset de medalhas (apenas linhas que têm medalha válida)
df_medals = df_all[df_all['medal'].notna()].copy()
# normalizar medal values (Gold/Silver/Bronze)
df_medals['medal'] = df_medals['medal'].str.title().str.strip()
df_medals = df_medals[df_medals['medal'].isin(['Gold','Silver','Bronze'])]


In [10]:
# Célula 7 — Função para contar medalhas por país e season
def medals_by_country(df_medals, season_filter=None):
    sub = df_medals.copy()
    if season_filter is not None:
        sub = sub[sub['season'] == season_filter]
    # preferir join by country code if exists (noc). Vamos usar 'country' tal como está.
    counts = sub.groupby(['country', 'medal']).size().unstack(fill_value=0)
    counts['Total'] = counts.sum(axis=1)
    counts = counts.sort_values('Total', ascending=False)
    counts = counts.reset_index().rename_axis(None, axis=1)
    return counts

medals_summer = medals_by_country(df_medals, season_filter='Summer')
medals_winter = medals_by_country(df_medals, season_filter='Winter')
medals_total = medals_by_country(df_medals, season_filter=None)

# Salvar tabelas em gold (parquet)
save_parquet(medals_summer, gold_a2 / 'medals_summer_top.parquet', metadata={'desc':'Medalhas - Jogos de Verão'})
save_parquet(medals_winter, gold_a2 / 'medals_winter_top.parquet', metadata={'desc':'Medalhas - Jogos de Inverno'})
save_parquet(medals_total, gold_a2 / 'medals_total_top.parquet', metadata={'desc':'Medalhas - Total Geral'})

# Também salvar CSVs para inspeção
medals_summer.to_csv(gold_a2 / 'medals_summer_top.csv', index=False)
medals_winter.to_csv(gold_a2 / 'medals_winter_top.csv', index=False)
medals_total.to_csv(gold_a2 / 'medals_total_top.csv', index=False)

medals_summer.head()


,country,Bronze,Gold,Silver,Total
0,USA,783,1116,885,2784
1,URS,323,431,343,1097
2,GBR,343,307,343,993
3,GER,329,273,297,899
4,FRA,306,243,279,828


In [11]:
def plot_top_n(df_counts, title, out_path, n=50):
    df = df_counts.copy()
    df_top = df.head(n).iloc[::-1]  # inverte para exibir da maior para a menor
    
    # 🔹 Ajuste automático da altura do gráfico com espaçamento mais largo
    bar_height = 0.8  # altura individual das barras (padrão é 0.8)
    spacing_factor = 1  # fator para aumentar o espaço entre barras
    fig_height = max(8, len(df_top) * bar_height * spacing_factor)
    
    fig, ax = plt.subplots(figsize=(16, fig_height))
    ax.barh(df_top['country'], df_top['Total'], height=bar_height)
    
    ax.set_xlabel('Total de medalhas', fontsize=12)
    ax.set_title(title, fontsize=16, pad=20)
    
    # 🔹 Aumenta o espaçamento entre os rótulos e melhora legibilidade
    ax.tick_params(axis='y', labelsize=10, pad=6)
    plt.tight_layout()
    
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

# Gráficos gerados
plot_top_n(medals_summer, 'Top 50 Países — Medalhas (Jogos de Verão)', gold_a2 / 'top50_summer.png', n=50)
plot_top_n(medals_winter, 'Top 50 Países — Medalhas (Jogos de Inverno)', gold_a2 / 'top50_winter.png', n=50)
plot_top_n(medals_total, 'Top 50 Países — Medalhas (Total Geral)', gold_a2 / 'top50_total.png', n=50)


# PARTE 2: Análise por Continente 

Precisamos de um mapa país → continente. O notebook usa estes fluxos:

Se existir raw/atividade2/country_continent.csv, usa-o.

Se estiver online e requests disponível, tenta baixar de um repositório público.

Fallback: tenta usar pycountry_convert (se instalado) para inferir continente a partir do alpha-2.

In [12]:
! pip install pycountry
! pip install pycountry-convert


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Célula 9 — Carregar/obter country -> continent
cc_local = raw_a2 / 'country_continent.csv'

def load_country_continent():
    if cc_local.exists():
        cc = pd.read_csv(cc_local, low_memory=False)
        # esperar colunas: country, continent (ou country_name, continent_name)
        cc.columns = [c.strip() for c in cc.columns]
        if 'country' not in cc.columns:
            # tentar lidar com nomes alternativos
            possible = [c for c in cc.columns if 'country' in c.lower()]
            if possible:
                cc = cc.rename(columns={possible[0]:'country'})
        if 'continent' not in cc.columns:
            possible = [c for c in cc.columns if 'continent' in c.lower()]
            if possible:
                cc = cc.rename(columns={possible[0]:'continent'})
        return cc[['country','continent']].drop_duplicates()
    # tentar baixar de uma fonte pública (exemplo: github gist com mapeamento)
    try:
        url = 'https://raw.githubusercontent.com/dbouquin/2016_CSV/master/countries.csv'
        cc = pd.read_csv(url)
        # esse arquivo tem colunas 'Country','Region' <- mapear
        if 'Country' in cc.columns and 'Region' in cc.columns:
            cc2 = cc[['Country','Region']].rename(columns={'Country':'country','Region':'continent'})
            return cc2
    except Exception:
        pass
    # fallback: tentar pycountry_convert (se instalado)
    try:
        import pycountry
        import pycountry_convert as pc
        countries = []
        for c in df_all['country_long'].dropna().unique():
            try:
                country_obj = pycountry.countries.lookup(c)
                alpha2 = country_obj.alpha_2
                continent_code = pc.country_alpha2_to_continent_code(alpha2)
                continent_name = {
                    'AF':'Africa','AS':'Asia','EU':'Europe','NA':'North America','OC':'Oceania','SA':'South America','AN':'Antarctica'
                }.get(continent_code, 'Other')
                countries.append({'country_long': c, 'continent': continent_name})
            except Exception:
                countries.append({'country_long': c, 'continent': None})
        return pd.DataFrame(countries)
    except Exception:
        raise RuntimeError("Não foi possível obter o mapeamento country->continent. Coloque country_continent.csv em raw/atividade2/")

cc_df = load_country_continent()
save_parquet(cc_df, bronze_a2 / 'country_continent.parquet', metadata={'desc':'mapeamento pais->continente'})
cc_df


,country_long,continent
0,Switzerland,Europe
1,France,Europe
2,Great Britain,None
3,Austria,Europe
4,Italy,Europe
...,...,...
725,Bosnia & Herzegovina,None
726,Lao PDR,None
727,DR Congo,None
728,Syria,Asia


In [14]:
# Célula 10 — Associar continente ao df_medals (por country). Cuidado: nomes podem divergir -> normalização simples
def normalize_country_name(s):
    if pd.isna(s): return s
    return str(s).strip().replace('United States of America','United States').replace('USA','United States').replace('U.S.A.','United States')

cc_df['country_norm'] = cc_df['country_long'].astype(str).map(lambda x: x.strip().lower())
df_medals['country_norm'] = df_medals['country_long'].astype(str).map(lambda x: str(x).strip().lower())

# tente primeiro join direto
df_medals_cont = df_medals.merge(cc_df[['country_norm','continent']], on='country_norm', how='left')

# se houver muitos nulos, tentar join por aproximação (fuzzy) — opcional, exige fuzzywuzzy (não obrigatório)
missing = df_medals_cont['continent'].isna().mean()
print(f"Percentual de país sem continente após join exato: {missing:.2%}")

# Salvar bronze
save_parquet(df_medals_cont, bronze_a2 / 'medals_with_continent.parquet', metadata={'desc':'medalhas com continente'})


Percentual de país sem continente após join exato: 23.42%


In [15]:
# Célula 11 — 2.1 Distribuição total de medalhas por continente (acumulado e por edição)
medals_continent_total = df_medals_cont.groupby('continent').size().reset_index(name='n_medals').sort_values('n_medals', ascending=False)
medals_continent_by_year = df_medals_cont.groupby(['year','continent']).size().reset_index(name='n_medals')

# Salvar
save_parquet(medals_continent_total, gold_a2 / 'medals_continent_total.parquet', metadata={'desc':'total por continente'})
save_parquet(medals_continent_by_year, gold_a2 / 'medals_continent_by_year.parquet', metadata={'desc':'por edição e continente'})

# Gráfico pizza (total acumulado)
plt.figure(figsize=(8,8))
plt.pie(medals_continent_total['n_medals'], labels=medals_continent_total['continent'], autopct='%1.1f%%', startangle=90)
plt.title('Distribuição acumulada de medalhas por continente (1986–2024)')
plt.savefig(gold_a2 / 'medals_continent_pie.png')
plt.close()

# Gráfico linha (por edição)
plt.figure(figsize=(12,6))
for cont in medals_continent_by_year['continent'].dropna().unique():
    sub = medals_continent_by_year[medals_continent_by_year['continent']==cont]
    plt.plot(sub['year'], sub['n_medals'], marker='o', label=cont)
plt.legend()
plt.title('Medalhas por continente por edição (ano)')
plt.xlabel('Ano')
plt.ylabel('Número de medalhas')
plt.tight_layout()
plt.savefig(gold_a2 / 'medals_continent_line_by_year.png')
plt.close()


In [18]:
df_all.country_long

0           Switzerland
1                France
2                France
3         Great Britain
4           Switzerland
              ...      
229247           Greece
229248              AIN
229249        DPR Korea
229250        DPR Korea
229251     South Africa
Name: country_long, Length: 229252, dtype: string

In [19]:
# Célula 12 — 2.1 Número médio de atletas por continente
# Para isso precisamos do número de atletas por país por edição -> então associar continente a df_all (não só medalhistas)
df_all = df_all.merge(cc_df[['country_long','continent']], on='country_long', how='left')

# Contar atletas únicos por continent x year (assumindo coluna 'athlete' com nome)
if 'athlete' in df_all.columns:
    athletes_by_continent_year = df_all.groupby(['year','continent'])['athlete'].nunique().reset_index(name='n_athletes')
    avg_athletes_continent = athletes_by_continent_year.groupby('continent')['n_athletes'].mean().reset_index(name='avg_athletes_per_edition')
    save_parquet(athletes_by_continent_year, gold_a2 / 'athletes_by_continent_year.parquet')
    save_parquet(avg_athletes_continent, gold_a2 / 'avg_athletes_continent.parquet')
else:
    athletes_by_continent_year = pd.DataFrame()
    avg_athletes_continent = pd.DataFrame()


In [22]:
# Célula 13 — 2.2 Crescimento da representação ao longo do tempo (média, desvio padrão, gráfico)
# Para cada continente, calcular média e std do número de atletas por edição
if not athletes_by_continent_year.empty:
    stats = athletes_by_continent_year.groupby('continent')['n_athletes'].agg(['mean','std','min','max']).reset_index()
    save_parquet(stats, gold_a2 / 'athletes_continent_stats.parquet')
    # gráfico de linha já foi gerado acima para medalhas; aqui gráfico de atletas por continente
    plt.figure(figsize=(12,6))
    for cont in athletes_by_continent_year['continent'].dropna().unique():
        p = athletes_by_continent_year[athletes_by_continent_year['continent']==cont]
        plt.plot(p['year'], p['n_athletes'], marker='o', label=cont)
    plt.legend()
    plt.title('Número de atletas por continente por edição')
    plt.savefig(gold_a2 / 'athletes_continent_line.png')
    plt.close()


In [23]:
# Célula 14 — 2.3 Participação feminina por continente
# Precisamos coluna 'sex' no df_all
if 'sex' in df_all.columns:
    df_sex = df_all[df_all['sex'].notna()].copy()
    df_sex['sex'] = df_sex['sex'].astype(str).str.upper().map(lambda x: 'F' if x.startswith('F') else ('M' if x.startswith('M') else x))
    female_share = df_sex.groupby(['year','continent','sex'])['athlete'].nunique().reset_index(name='n_athletes')
    # pivot para proporções
    pivot = female_share.pivot_table(index=['year','continent'], columns='sex', values='n_athletes', fill_value=0).reset_index()
    if 'F' in pivot.columns and 'M' in pivot.columns:
        pivot['prop_f'] = pivot['F'] / (pivot['F'] + pivot['M'])
    else:
        pivot['prop_f'] = np.nan
    save_parquet(pivot, gold_a2 / 'female_share_continent_year.parquet')
    # gráfico: linhas por continente
    plt.figure(figsize=(12,6))
    for cont in pivot['continent'].dropna().unique():
        s = pivot[pivot['continent']==cont]
        plt.plot(s['year'], s['prop_f'], marker='o', label=cont)
    plt.legend()
    plt.title('Proporção feminina por continente ao longo do tempo')
    plt.ylabel('Proporção F')
    plt.savefig(gold_a2 / 'female_prop_continent_line.png')
    plt.close()
else:
    print("Coluna 'sex' não encontrada em df_all; não foi possível calcular participação feminina.")


In [24]:
# Célula 15 — 2.4 Modalidades mais fortes por continente (top N)
# Agrupar medalhas por continent x sport
medals_by_cont_sport = df_medals_cont.groupby(['continent','sport']).size().reset_index(name='n_medals')
# Para cada continente, pegar top 10 esportes
top_by_cont = medals_by_cont_sport.sort_values(['continent','n_medals'], ascending=[True,False]).groupby('continent').head(10)
save_parquet(medals_by_cont_sport, gold_a2 / 'medals_by_continent_sport.parquet')
top_by_cont.to_csv(gold_a2 / 'top_sports_by_continent.csv', index=False)

# Example plot for a continent (e.g., 'Europe')
for cont in top_by_cont['continent'].dropna().unique():
    dfc = top_by_cont[top_by_cont['continent']==cont].sort_values('n_medals')
    plt.figure(figsize=(10, max(4, 0.3*len(dfc))))
    plt.barh(dfc['sport'], dfc['n_medals'])
    plt.title(f'Top sports by medals — {cont}')
    plt.tight_layout()
    plt.savefig(gold_a2 / f'top_sports_{cont}.png')
    plt.close()


In [25]:
# Célula 16 — 2.5 Crescimento nas medalhas entre 1986 e 2024 por continente
# calcular média histórica (1986-2016 por exemplo) vs Paris 2024
base_period = (df_medals_cont['year'] >= 1986) & (df_medals_cont['year'] < 2024)
mean_hist = df_medals_cont[base_period].groupby('continent').size().reset_index(name='mean_historical')
paris24 = df_medals_cont[df_medals_cont['year']==2024].groupby('continent').size().reset_index(name='paris_2024')
growth = mean_hist.merge(paris24, on='continent', how='outer').fillna(0)
growth['absolute_change'] = growth['paris_2024'] - growth['mean_historical']
growth['relative_pct'] = growth['absolute_change'] / growth['mean_historical'].replace(0, np.nan) * 100
growth = growth.sort_values('absolute_change', ascending=False)
save_parquet(growth, gold_a2 / 'continent_medal_growth_1986_2024.parquet')
growth.to_csv(gold_a2 / 'continent_medal_growth_1986_2024.csv', index=False)


In [26]:
# Célula 17 — Escrever notebooks / metadata / resumo final
# Salvar um metadata summary simples
meta_summary = {
    'medals_summer': str(gold_a2 / 'medals_summer_top.parquet'),
    'medals_winter': str(gold_a2 / 'medals_winter_top.parquet'),
    'medals_total': str(gold_a2 / 'medals_total_top.parquet'),
    'medals_continent_total': str(gold_a2 / 'medals_continent_total.parquet')
}
import json
(meta_a2 / 'metadata_summary.json').write_text(json.dumps(meta_summary, indent=2, ensure_ascii=False))

print("Pipeline Atividade2 concluído. Verifique pasta gold/atividade2 para resultados (parquet + csv + gráficos).")


Pipeline Atividade2 concluído. Verifique pasta gold/atividade2 para resultados (parquet + csv + gráficos).
